IMPORTING LIBRARIES FOR CREATING MODEL

In [8]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, Dropout
from tensorflow.keras.optimizers import SGD



CREATING DATASETS FOR TRAINING AND VALIDATION

In [66]:
training_datagen = ImageDataGenerator(rescale = 1./255,horizontal_flip=True,rotation_range=45,height_shift_range=0.2,fill_mode='nearest')
validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory("fire_dataset",target_size=(224,224),class_mode='categorical',batch_size = 64)
validation_generator = validation_datagen.flow_from_directory("Fire-Detection",target_size=(224,224),class_mode='categorical',batch_size= 16)

Found 1180 images belonging to 2 classes.
Found 651 images belonging to 2 classes.


CREATING INCEPTIONV3 MODEL FOR IMAGE RECOGNITION AND CLASSIFICATION OF FIRE AND NONE FIRE IMAGE.
OUR MODEL CONTAINS 1 INPUT LAYER(GLOBALAVERAGEPOOLING2D), 3 DENSE LAYERS AND 3 DROPOUT AYERS AND 1 OUTPUT LAYER. 

In [67]:
input_tensor = Input(shape=(224, 224, 3))
base_model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(2048, activation='relu')(x)
x = Dropout(0.25)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(2, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
  layer.trainable = False
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
history = model.fit(train_generator,steps_per_epoch = 14,epochs = 20,validation_data = validation_generator,validation_steps = 14)

#To train the top 2 inception blocks, freeze the first 249 layers and unfreeze the rest.
for layer in model.layers[:249]:
  layer.trainable = False
for layer in model.layers[249:]:
  layer.trainable = True
#Recompile the model for these modifications to take effect
from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(learning_rate=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['acc'])
history = model.fit(train_generator,steps_per_epoch = 14,epochs = 10,validation_data = validation_generator,validation_steps = 14)

Epoch 1/20
14/14 [==============================] - 41s 3s/step - loss: 5.2174 - acc: 0.6105 - val_loss: 0.2786 - val_acc: 0.9152
Epoch 2/20
14/14 [==============================] - 40s 3s/step - loss: 0.4194 - acc: 0.8035 - val_loss: 0.2265 - val_acc: 0.9196
Epoch 3/20
14/14 [==============================] - 41s 3s/step - loss: 0.3227 - acc: 0.8627 - val_loss: 0.1529 - val_acc: 0.9688
Epoch 4/20
14/14 [==============================] - 41s 3s/step - loss: 0.2720 - acc: 0.8942 - val_loss: 0.1474 - val_acc: 0.9330
Epoch 5/20
14/14 [==============================] - 42s 3s/step - loss: 0.2359 - acc: 0.9151 - val_loss: 0.2006 - val_acc: 0.9107
Epoch 6/20
14/14 [==============================] - 44s 3s/step - loss: 0.2698 - acc: 0.8965 - val_loss: 0.1874 - val_acc: 0.9554
Epoch 7/20
14/14 [==============================] - 46s 3s/step - loss: 0.2474 - acc: 0.9058 - val_loss: 0.1505 - val_acc: 0.9420
Epoch 8/20
14/14 [==============================] - 46s 3s/step - loss: 0.2025 - acc: 0.92

APPLICATION OF OUR MODEL ON LIVE FEED.

In [119]:
import cv2
import numpy as np
from PIL import Image
import tensorflow as tf
from playsound import playsound
from keras.preprocessing import image
from twilio.rest import Client
#Load the saved model
video = cv2.VideoCapture(0)

for i in range(10):
        l=[]
        for j in range(10):
                _,frame = video.read()
                im = Image.fromarray(frame,'RGB')
                im = im.resize((224,224))
                img_array = image.img_to_array(im)
                img_array = np.expand_dims(img_array, axis=0) / 255
                probabilities = model.predict(img_array)[0]
                prediction = np.argmax(probabilities)
                l.append(prediction)
        #frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        
        #if prediction == 0:
                
                #playsound('Fire alarm - 2018.mp3')
                #print('playing sound using  playsound')
        if l.count(1)<=2:
                playsound('Fire alarm - 2018.mp3')
                client = Client('ACe524474472b65c3c14d2d94a59e0b83c', 'c96c2963972f5868db82e7ac2bf3d53b')
                message = client.calls.create(
                     from_='+19704898322',
                     to='+918368751110',
                     url = 'https://api.twilio.com/2010-04-01/Accounts/ACe524474472b65c3c14d2d94a59e0b83c/Calls.json' )
                
                
                
                print('fire')
                
                
                
        else:
                print('chill hai')
        cv2.imshow("Capturing", frame)
        key=cv2.waitKey(1)
        if key == ord('q'):
                break
video.release()
cv2.destroyAllWindows()

fire
chill hai
chill hai
chill hai
chill hai
chill hai
chill hai
chill hai
chill hai
chill hai
